<a href="https://colab.research.google.com/github/sukantjain/Python/blob/main/IMDLIB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Script to download IMD Gridded data into CSV file**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install imdlib

In [ ]:
import imdlib as imd

path = "/content/drive/MyDrive/Colab Notebooks"

start_yr = 2019
end_yr = 2020
variable = 'rain' # other options are ('tmin'/ 'tmax')

imd.get_data(variable, start_yr, end_yr, fn_format='yearwise', file_dir=path)
data = imd.open_data(variable, start_yr, end_yr,'yearwise', path)
ds = data.get_xarray()
print(ds)



lat = 20.03 #lattitude of point
lon = 77.23 #longitude of point
data.to_csv('data.csv', lat, lon, path)

In [ ]:
# Save CSV files for multiple points

# Provide lat and long in a list
latLong = [[20.3,77.23],[23.5,72.5],[26.0,77,1]]

for points in latLong:
  lat = points[0]
  lon = points[1]

  data.to_csv('test.csv', lat, lon, path)
  print ("data save for ",points)